In [1]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
# os.environ['NUMEXPR_MAX_THREADS'] = '48'
# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
# utils
from ppsplit.utils import create_dir

/home/dengruijun/miniconda3/envs/drj-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
Author: Ruijun Deng
Date: 2024-08-24 00:41:28
LastEditTime: 2024-10-27 02:06:57
LastEditors: Ruijun Deng
FilePath: /PP-Split/target_model/models.ipynb
Description: 
'''
# 导入各个baseline模型及其数据集预处理方法
# 模型
from target_model.utils import split_weights_client
from target_model.models.ImageClassification.VGG5_9 import VGG,VGG5Decoder,model_cfg
from target_model.models.TableClassification.BankNet import BankNet1,BankNetDecoder1,bank_cfg
from target_model.models.TableClassification.CreditNet import CreditNet1,CreditNetDecoder1,credit_cfg
from target_model.models.TableClassification.PurchaseNet import PurchaseClassifier1,PurchaseDecoder1,purchase_cfg
from target_model.models.TableClassification.IrisNet import IrisNet,IrisNetDecoder,Iris_cfg
from target_model.models.ImageClassification.ResNet import resnet18,resnet34,resnet50,InversionNet,resnet_model_cfg
from target_model.models.ImageClassification.AlexNet import AlexNet_MNIST
from target_model.models.ImageClassification.ViT import vit_b_16 as ViTb_16


# 数据预处理方法
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,deprocess
from target_model.data_preprocessing.preprocess_bank import bank_dataset,preprocess_bank,preprocess_bank_dataset,tabinfo_bank
from target_model.data_preprocessing.preprocess_credit import preprocess_credit,tabinfo_credit
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase,tabinfo_purchase
from target_model.data_preprocessing.preprocess_Iris import preprocess_Iris, tabinfo_Iris
from target_model.data_preprocessing.dataset import get_one_data


# utils
from  ppsplit.utils import create_dir


# model_select
from target_model.task_select import get_dataloader_and_model,get_dataloader_and_model, \
    get_dataloader,get_models,get_infotopo_para

In [3]:

args =  {
        "result_dir":"20241228-defense/",
        "test_num":"Posthoc",
        "device":"cuda:0",

        "dataset":"CIFAR10",
        "oneData_bs": 1,
        "train_bs": 32,
        "test_bs": 64,
        
        "model":"VGG5",
        "split_layer": 2,
        "ep":-1,
        "no_dense": 0,
        
        "noise_scale": 0
    }
print(args['device'])
print(args)

cuda:0
{'result_dir': '20241228-defense/', 'test_num': 'Posthoc', 'device': 'cuda:0', 'dataset': 'CIFAR10', 'oneData_bs': 1, 'train_bs': 32, 'test_bs': 64, 'model': 'VGG5', 'split_layer': 2, 'ep': -1, 'no_dense': 0, 'noise_scale': 0}


# 1. ResNet18

In [4]:
# resnet18
net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')
print(net)

Num unit layers: 14
Split layer: -1
ResNet(
  (selected_layers): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate=none)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=1)
    (4): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU

In [ ]:
# decoder
# inet = InversionNet(in_c=64, upconv_channels=[(128, 'same'), (3, 'same')], last_activation=None)
inet = InversionNet(last_activation=None)
print(inet)

In [10]:
# 加载一下模型参数
# resnet18 cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/CIFAR10-models/ResNet18/32bs-ep20-relu-max-adam/resnet18-drj.pth' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnet18 cifar100

# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet34 cifar100
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj-align.pth'
# client_net = resnet34(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet50 cifar100
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj-align.pth'
# # client_net = resnet50(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet18 20ep cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/epoch=14resnet18-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')


# resnet18 20ep narrow cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/epoch=16resnet18_narrow-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnett18 20ep wide cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/epoch=14resnet18_wide-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnet34 20ep cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR10/epoch=10-args.classifier=0-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet34(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnet18 20ep 2narrow cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_2narrow/CIFAR10/epoch=14resnet18_2narrow-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_2narrow/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_2narrow/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# maeng 已训练好的参数 resnet18,34,50
# 18
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet18.pt'
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet34.pt'
new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet50.pt'

# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')
# client_net = resnet34(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')
client_net = resnet50(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='relu', pooling='max')

# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet18-align.pt'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet34-align.pt'
new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ImageClassification/CIFAR10-models/state_dicts/resnet50-align.pt'


Num unit layers: 22
Split layer: -1


In [ ]:
# 提取state_dict
# pweights = torch.load(unit_net_route)
# print(pweights.keys())
# state_dict = pweights['state_dict']
# torch.save(state_dict, new_unit_net_route)



# 改一下 resnet的 模型参数的key
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
import torch

# from target_model.task_select import *

pweights = torch.load(new_unit_net_route) # 存的参数
cweights = client_net.state_dict() # 当前模型的参数名称

print(len(pweights.keys()),pweights.keys())
print(len(cweights.keys()),cweights.keys())


# new_key = {}
# for keyp,keyc in zip(pweights.keys(),cweights.keys()):
#     print(keyp,'\t\t\t',keyc)

# for i,key in enumerate(pweights.keys()):
#     if i<122:
#         continue
#     print(key)

    # new_key[key.replace('model.','')] = pweights[key]

# new_key = {}
# for key in cweights.keys():
#     # new_key[key] = pweights[key.replace('selected_layers','model.layers')] # resnet18 cifar10
#     # new_key[key] = pweights[key.replace('selected_layers','model.selected_layers')] # resnet18 cifar100
#     new_key[key] = pweights[key.replace('selected_layers','model.selected_layers')] # resnet18 cifar100
# torch.save(new_key, new_unit_net_route_align)

# 每个元素使用第一个字典的value和第二个字典的key
# new_key = {key2: value1 for value1, key2 in zip(pweights.values(), cweights.keys())}
# torch.save(new_key, new_unit_net_route_align)

320 odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv

In [6]:
# 加载模型参数
# client_net = resnet18(pretrained=False, split_layer=13, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')
state_dict = torch.load(new_unit_net_route_align)
client_net.load_state_dict(state_dict) 
print(client_net)

ResNet(
  (selected_layers): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate=none)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=1)
    (4): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Co

# 2. AlexNet

In [4]:
# alexnet mnist
net = AlexNet_MNIST()
print(net)
# print(len(net)) 

AlexNet_MNIST(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer5): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
  )
  (fc1): Linear(in_features=2304, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (

# 3. VGG9

In [12]:
# CIFAR-10
# model_msg = get_models(args)
# # 模型
# client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
# decoder_route = model_msg['decoder_route']
# image_deprocess = model_msg['image_deprocess']


client_net = VGG('Unit','VGG9',len(model_cfg['VGG9'])-1,model_cfg,noise_scale=args['noise_scale'])
print(client_net)
# 0ep
# client_net.load_state_dict(torch.load('/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG9/CIFAR10/VGG9-CIFAR10-20epoch.pth-0.pth'))
# 20ep
client_net.load_state_dict(torch.load('/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG9/CIFAR10/VGG9-CIFAR10-20epoch.pth-20.pth'))

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Tanh()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Tanh()
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

<All keys matched successfully>

In [8]:
# MNIST
client_net = VGG('Unit','VGG9_MNIST',len(model_cfg['VGG9'])-1,model_cfg,noise_scale=args['noise_scale'])
print(client_net)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Tanh()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Tanh()
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

# 4. VGG5

In [7]:
# CIFAR10
client_net = VGG('Unit','VGG5',2,model_cfg,noise_scale=args['noise_scale'])
print(client_net)

# 0ep
pweights=torch.load('/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/CIFAR10/VGG5-CIFAR10-0epoch.pth')
# 20ep
pweights=torch.load('/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/CIFAR10/VGG5-CIFAR10-20epoch.pth')


pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)
print(client_net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Tanh()
  )
  (denses): Sequential(
    (0): Linear(in_features=4096, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=10, bias=True)
  )
)
features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.

In [4]:
# MNIST
client_net = VGG('Unit','VGG5_MNIST',2,model_cfg,noise_scale=args['noise_scale'])
print(client_net)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Tanh()
  )
  (denses): Sequential(
    (0): Linear(in_features=3136, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=10, bias=True)
  )
)


# 6. ViT

In [4]:
# 导入torch官网参数
from torchvision.models import vit_b_16,ViT_B_16_Weights
import os
os.environ['TORCH_HOME'] = '/home/dengruijun/data/project/data/torch_models/'
vit_model = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
print(vit_model)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

In [4]:
# 查看官网参数内容
unit_net_route = '/home/dengruijun/data/project/data/torch_models/hub/checkpoints/vit_b_16-c867db91.pth'
weights = torch.load(unit_net_route)
print(weights.keys())
# print(weights.state_dict().keys())

odict_keys(['class_token', 'conv_proj.weight', 'conv_proj.bias', 'encoder.pos_embedding', 'encoder.layers.encoder_layer_0.ln_1.weight', 'encoder.layers.encoder_layer_0.ln_1.bias', 'encoder.layers.encoder_layer_0.self_attention.in_proj_weight', 'encoder.layers.encoder_layer_0.self_attention.in_proj_bias', 'encoder.layers.encoder_layer_0.self_attention.out_proj.weight', 'encoder.layers.encoder_layer_0.self_attention.out_proj.bias', 'encoder.layers.encoder_layer_0.ln_2.weight', 'encoder.layers.encoder_layer_0.ln_2.bias', 'encoder.layers.encoder_layer_0.mlp.linear_1.weight', 'encoder.layers.encoder_layer_0.mlp.linear_1.bias', 'encoder.layers.encoder_layer_0.mlp.linear_2.weight', 'encoder.layers.encoder_layer_0.mlp.linear_2.bias', 'encoder.layers.encoder_layer_1.ln_1.weight', 'encoder.layers.encoder_layer_1.ln_1.bias', 'encoder.layers.encoder_layer_1.self_attention.in_proj_weight', 'encoder.layers.encoder_layer_1.self_attention.in_proj_bias', 'encoder.layers.encoder_layer_1.self_attention.o

In [5]:
# 用我自己改的ViT模型代码
vit_model = ViTb_16()
print(vit_model)
vit_model.load_state_dict(weights) # 可以直接load 官网参数，没有问题。
print(vit_model.state_dict().keys())

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

In [19]:
# 切割后模型 参数加载
unit_net_route = '/home/dengruijun/data/project/data/torch_models/hub/checkpoints/vit_b_16-c867db91-drj.pth'
split_layer = 13
pweights = torch.load(unit_net_route)
vit_model = ViTb_16(split_layer=split_layer)
print(vit_model)

if split_layer < 13:
    pweights = split_weights_client(pweights, vit_model.state_dict())
vit_model.load_state_dict(pweights,strict=False)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

<All keys matched successfully>

In [13]:
# 模型参数key修改： linear_1.weight -> 3.weight
unit_net_route = '/home/dengruijun/data/project/data/torch_models/hub/checkpoints/vit_b_16-c867db91.pth'
new_unit_net_route_align = '/home/dengruijun/data/project/data/torch_models/hub/checkpoints/vit_b_16-c867db91-drj.pth'
# split_layer = 1
pweights = torch.load(unit_net_route)
vit_model = ViTb_16()
cweights = vit_model.state_dict() 

new_key = {}
for key in cweights.keys():
    print(key)
    print(key.replace('mlp.0','mlp.linear_1'))
    if('mlp.0' in key):
        new_key[key] = pweights[key.replace('mlp.0','mlp.linear_1')]
    elif('mlp.3' in key):
        new_key[key] = pweights[key.replace('mlp.3','mlp.linear_2')]
    else:
        new_key[key] = pweights[key]
torch.save(new_key, new_unit_net_route_align)

new_key = {}
for keyp,keyc in zip(pweights.keys(),vit_model.state_dict().keys()):
    print(keyp,'\t\t\t',keyc)


# for i in range(2):
#     for type in ["weight", "bias"]:
#         old_key = f"{prefix}linear_{i+1}.{type}"
#         new_key = f"{prefix}{3*i}.{type}"
#         if old_key in state_dict:
#             state_dict[new_key] = state_dict.pop(old_key)


class_token
class_token
conv_proj.weight
conv_proj.weight
conv_proj.bias
conv_proj.bias
encoder.pos_embedding
encoder.pos_embedding
encoder.layers.encoder_layer_0.ln_1.weight
encoder.layers.encoder_layer_0.ln_1.weight
encoder.layers.encoder_layer_0.ln_1.bias
encoder.layers.encoder_layer_0.ln_1.bias
encoder.layers.encoder_layer_0.self_attention.in_proj_weight
encoder.layers.encoder_layer_0.self_attention.in_proj_weight
encoder.layers.encoder_layer_0.self_attention.in_proj_bias
encoder.layers.encoder_layer_0.self_attention.in_proj_bias
encoder.layers.encoder_layer_0.self_attention.out_proj.weight
encoder.layers.encoder_layer_0.self_attention.out_proj.weight
encoder.layers.encoder_layer_0.self_attention.out_proj.bias
encoder.layers.encoder_layer_0.self_attention.out_proj.bias
encoder.layers.encoder_layer_0.ln_2.weight
encoder.layers.encoder_layer_0.ln_2.weight
encoder.layers.encoder_layer_0.ln_2.bias
encoder.layers.encoder_layer_0.ln_2.bias
encoder.layers.encoder_layer_0.mlp.0.weight
enco

In [4]:
# 看看面相其他数据集的模型大小
# cifar10
vit_model = ViTb_16()
print(vit_model)
torch.save(vit_model.state_dict(),'/home/dengruijun/data/project/data/torch_models/hub/checkpoints/vit_b_16-cifar10-8*8*3.pth')


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attenti

# 4. 模型信息计算

In [36]:
import torch
import torch.nn as nn


# 计算模型参数的平均第二范数
def average_l2_norm(model):
    total_l2_norm = 0.0
    num_params = 0

    for param in model.parameters():
        if param.requires_grad:
            l2_norm = torch.norm(param, p=2)
            total_l2_norm += l2_norm.item()
            num_params += 1

    avg_l2_norm = total_l2_norm / num_params if num_params > 0 else 0.0
    return avg_l2_norm

def average_l2_norm_state(state_dict):
    total_l2_norm = 0.0
    num_params = 0

    for param_tensor in state_dict.values():
        if param_tensor.requires_grad:
            l2_norm = torch.norm(param_tensor, p=2)
            total_l2_norm += l2_norm.item()
            num_params += 1
    avg_l2_norm = total_l2_norm / num_params if num_params > 0 else -1
    return avg_l2_norm

# 打印平均第二范数
# model
avg_l2_norm = average_l2_norm(client_net)
# avg_l2_norm = average_l2_norm_state(pweights)
print(f"模型参数的平均第二范数: {avg_l2_norm}")
# print(pweights.values())

模型参数的平均第二范数: 7.32628158852458


In [3]:
# 计算模型参数数目
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 计算模型每层输出的大小
def model_output_size(model, input):
    input = torch.randn_like(input)
    output_feat = model(input)
    return output_feat.size()

In [7]:
# 测试vit
vit_model = ViTb_16(split_layer=13)

model_param_num = count_parameters(vit_model)
print(f"模型参数数目: {model_param_num}")

for i in range(13):
    vit_model = ViTb_16(split_layer=i)
    print(f"第{i}层输出的大小: {model_output_size(vit_model, torch.randn(1, 3, 8, 8))}")

模型参数数目: 85975528
第0层输出的大小: torch.Size([1, 1, 768])
第1层输出的大小: torch.Size([1, 2, 768])
第2层输出的大小: torch.Size([1, 2, 768])
第3层输出的大小: torch.Size([1, 2, 768])
第4层输出的大小: torch.Size([1, 2, 768])
第5层输出的大小: torch.Size([1, 2, 768])
第6层输出的大小: torch.Size([1, 2, 768])
第7层输出的大小: torch.Size([1, 2, 768])
第8层输出的大小: torch.Size([1, 2, 768])
第9层输出的大小: torch.Size([1, 2, 768])
第10层输出的大小: torch.Size([1, 2, 768])
第11层输出的大小: torch.Size([1, 2, 768])
第12层输出的大小: torch.Size([1, 2, 768])


# task select获取

In [4]:
data_msg = get_dataloader(args)
model_msg = get_models(args)

unit_net weights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked', 'features.8.weight', 'features.8.bias', 'features.9.weight', 'features.9.bias', 'features.9.running_mean', 'features.9.running_var', 'features.9.num_batches_tracked', 'denses.0.weight', 'denses.0.bias', 'denses.1.weight', 'denses.1.bias'])
client_net cweights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked'])
len unit_net weights:  25
